# Pittsburgh Parking Meters by Neighborhood

In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

parking = pd.read_csv("./parking_meters.csv")

neighborhoods = gpd.read_file("../neighborhood/Neighborhoods_.shp")
parking_locs = gpd.read_file("./pghdb_sde_PPA_ParkingMeters.shp")

## An overlay of metered parking locations in Pittsburgh

In [ ]:
pgh = neighborhoods.plot(cmap="OrRd")
parking_locs.plot(ax=pgh, color="blue", markersize=5)

## Number of metered parking locations grouped by neighborhood

In [ ]:
spots_per_neighborhood = parking.groupby("node").count().sort_values("fid", ascending=False)
spots_per_neighborhood.rename(columns={'fid':'count'}, inplace=True)
spots_per_neighborhood

## Top 10 neighborhoods with the most parking locations

In [ ]:
top_10_by_count = spots_per_neighborhood.head(10)
top_10_by_count

## Parking locations ranked by cheapest price

In [ ]:
price_ranked = parking.sort_values("rates").dropna() # Drop NaN since some are missing data
price_ranked

## Top 10 ranked by cheapest price

In [ ]:
top_10_by_price = price_ranked.head(10)
top_10_by_price

## Average price per neighborhood

In [ ]:
def calc_avg(set):
    sum = 0
    for rate in set:
        sum += float(str(rate).strip("$"))
    return sum / len(set)

parking["node"].replace(' ', np.nan, inplace=True) # Replaces missing data with NaN to get dropped by dropna()

avg_neighborhood_price = parking.groupby("node")["rates"].apply(calc_avg).dropna().sort_values().to_frame()
avg_neighborhood_price

## Top 10 cheapest neighborhoods

In [ ]:
top_10_cheapest_neighborhoods = avg_neighborhood_price.head(10)
top_10_cheapest_neighborhoods

## Cheapest neighborhoods with most metered locations

In [ ]:
best_neighborhoods = top_10_cheapest_neighborhoods.join(other=top_10_by_count, how="inner", lsuffix="", rsuffix="_")
best_neighborhoods[["rates", "count"]]